# Tipping Module Calibration
---
**Author** : Amaury Laridon

**Date** : 28/10/23

---

## Algorithme I with parameterization choice A for the fresh water flux

Here, we will numerically translate Algorithm I, which corresponds to the method described in [[Note de Travail - Cadre mathématique coefficients de couplage]] and which is a generalized application of Monteiro's (2022) method. We first look at the simplest choice of parameterization, i.e. choice "A" for the freshwater flow with a single forcing parameter.

### AMOC's calibration

#### Reminder of the mathematical framework
We wish to calibrate the following ODE
$$   \frac{d\Psi}{dt} = -\Psi^3 + a_1\Psi^2 + b_1\Psi + c_1 + d_1 T + e_{12} F_{GIS}
\quad (9)   $$
The mathematical framework described in [[Note de Travail - Cadre mathématique coefficients de couplage]] has given us the following results for coefficient values,

$$a_1 = \frac{3(\Psi^{-}+\Psi^{+})}{2} \quad (1) $$
$$b_1 =  -3 \Psi^{-}\Psi^+ \quad (2)$$
$$  c_1^A  = \frac{T^+_\Psi\Psi^{-^{2}}(\Psi^{-}-3\Psi^+)-T^{-}_\Psi\Psi^{+^{2}}(\Psi^{+}-3\Psi^{-})}{2(T^{-}_\Psi-T^{+}_\Psi)} \quad (3) $$
$$  d_1 = -\frac{(\Psi^{+}-\Psi^{-})^3}{2(T^{+}_\Psi-T^{-}_\Psi)} \quad (4)   $$
$$ c_1^B  = \frac{F_{GIS}^+\Psi^{-^{2}}(\Psi^{-}-3\Psi^+)-F_{GIS}^{-}\Psi^{+^{2}}(\Psi^{+}-3\Psi^{-})}{2(F_{GIS}^{-}-F_{GIS}^{+})} \quad (5) $$
$$  e_{12} =-\frac{(\Psi^{+}-\Psi^{-})^3}{2(F_{GIS}^{+}-F_{GIS}^{-})} \quad (6) $$
$$ c_1 = c_1^A - e_{12}F_{GIS}^A \quad (7) $$
$$c_1 = c_1^B - d_1T_{\Psi}^B \quad (8) $$

#### Numerical implementation

First, we want to make a numerical implementation of this mathematical framework and test it by giving as input any critical values of the bifurcation coordinates. In other words, we're first going to see whether, given the data we want to reproduce, the algorithm is a good fit before, in a second phase, providing the algorithm with real experimental values as input.

##### Libraries import 

In [2]:
import matplotlib.pyplot as plt
import numpy as np

##### Calibration module 

The critical values of the system variable under study (AMOC or GIS) are generally denoted by $xp, xm$, corresponding to the upper and lower critical values respectively. The critical values of the forcings are noted $op, om$. 

To avoid confusion, I explicitly define a function by coefficients, although the form is the same for many coefficients and I can simply change the arguments of the common function when calling the function and assigning it to the coefficient.

In [6]:
### Definition of the functions computing the coefficient's values. 
def a1(xp,xm):
    """Definition of the a_1 coefficient cfr. Eq.(1)"""
    return (3/2)*(xp+xm)
def b1(xp,xm):
    """Definition of the b_1 coefficient cfr. Eq.(2)"""
    return -3*xp*xm
def c1_A(xp,xm,op,om):
    """Definition of the c_1^A coefficient cfr. Eq.(3)"""
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))
def d1(xp,xm,op,om):
    """Definition of the d_1 coefficient cfr. Eq.(4)"""
    return -(xp-xm)**3/(2*(op-om))
def c1_B(xp,xm,op,om):
    """Definition of the c_1^B coefficient cfr. Eq.(5)"""
    return (op*xm**2*(xm-3*xp)-om*xp**2*(xp-3*xm))/(2*(om-op))
def e12(xp,xm,op,om):
    """Definition of the c_12 coefficient cfr. Eq.(6)"""
    return -(xp-xm)**3/(2*(op-om))
def c1_from_A(c1_A, e12, F_A):
    """Definition of the c_1 coefficient from experience A cfr. Eq.(7)"""
    return c1_A-(e12*F_A)
def c1_from_B(c1_B, e12, T_B):
    """Definition of the c_1 coefficient from experience B cfr. Eq.(8)"""
    return c1_B-(e12*T_B)
### Definition of the calibration module ###
def calib_module_AMOC_ParamA(psip,psim, Tp, Tm, Fp, Fm, F_A, T_B):
    """Calibration module for the AMOC with the parametrisation A for the fresh water flux."""
    a_1 = a1(psip, psim)
    b_1 = b1(psip,psim)
    c1A = c1_A(psip, psim, Tp, Tm)
    d_1 = d1(psip,psim, Tp, Tm)
    c1B = c1_B(psip, psim, Fp, Fm)
    e_12 = e12(psip, psim, Fp, Fm)
    c1_f_A = c1_from_A(c1A, e_12, F_A)
    c1_f_B = c1_from_B(c1B, d_1, T_B)
    return [a_1, b_1, c1A, d_1, c1B, e_12, c1_f_A, c1_f_B]


##### Test data

In [8]:
psip = 1
psim = 0.1
Tp = 3
Tm = 1.2
Fp = 0.5
Fm = 0.05
F_A = 0.1
T_B = 2

##### Algorithm's test

In [9]:
calibrate_coef = calib_module_AMOC_ParamA(psip, psim, Tp, Tm, Fp, Fm, F_A, T_B)

